<a href="https://colab.research.google.com/github/SMotaee/Deep-Learning/blob/main/Handling%20cold%20start%20problems%20in%20recommender%20system%20through%20Generative%20Adversarial%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Create the user-item matrix
num_users = data.user_id.nunique()
num_movies = data.movie_id.nunique()
user_item_matrix = np.zeros((num_users, num_movies))

for row in train_data.itertuples():
    user_item_matrix[row[1]-1, row[2]-1] = row[3]


In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set the dimension of the generator's input
gen_input_dim = 32

# Define the generator model
gen_input = Input(shape=(gen_input_dim,))
x = Dense(64)(gen_input)
x = LeakyReLU(alpha=0.2)(x)
x = Dense(num_movies)(x)
gen_output = LeakyReLU(alpha=0.2)(x)
generator = Model(gen_input, gen_output)

# Define the discriminator model
disc_input = Input(shape=(num_movies,))
x = Dense(64)(disc_input)
x = LeakyReLU(alpha=0.2)(x)
x = Dropout(0.5)(x)
disc_output = Dense(1, activation='sigmoid')(x)
discriminator = Model(disc_input, disc_output)

# Compile the discriminator model
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Define the GAN model
gan_input = Input(shape=(gen_input_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

# Compile the GAN model
gan.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Train the GAN model
epochs = 10000
batch_size = 32
half_batch_size = batch_size // 2

for epoch in range(epochs):
    # Select a random batch of real user-item ratings
    real_indices = np.random.randint(0, num_users, half_batch_size)
    real_ratings = user_item_matrix[real_indices]
    # Generate a random batch of noise as the generator's input
    noise = np.random.normal(0, 1, (half_batch_size, gen_input_dim))
    # Generate a batch of synthetic user-item ratings using the generator
    fake_ratings = generator.predict(noise)
    # Concatenate the real and fake ratings to create a batch of training data for the discriminator
    disc_input = np.concatenate([real_ratings, fake_ratings])
    # Create the labels for the discriminator
    disc_labels = np.concatenate([np.ones


In [ ]:
# Generate a batch of synthetic user-item ratings for the test set using the generator
test_noise = np.random.normal(0, 1, (len(test_data), gen_input_dim))
synthetic_test_ratings = generator.predict(test_noise)

# Compute the mean squared error (MSE) between the synthetic test ratings and the actual test ratings
mse = ((synthetic_test_ratings - test_data.rating.values.reshape(-1, 1))**2).mean()

print("MSE:", mse)